<a href="https://colab.research.google.com/github/spalmpppy/W3_HW_ML_6610412004_Chayapon/blob/main/HW_ML_6610412004_Chayapon_Meunchang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pickle
import matplotlib.pyplot as plt

df_price_housing = pd.read_csv("https://raw.githubusercontent.com/spalmpppy/W3_HW_ML_6610412004_Chayapon/main/Housing.csv",sep=",")
df_price_housing

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


In [2]:
#Survey data and check missing
df_price_housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 55.5+ KB


In [3]:
# Check for unique values in specific columns
print(df_price_housing['mainroad'].unique())
print(df_price_housing['guestroom'].unique())
print(df_price_housing['basement'].unique())
print(df_price_housing['hotwaterheating'].unique())
print(df_price_housing['airconditioning'].unique())
print(df_price_housing['parking'].unique())
print(df_price_housing['prefarea'].unique())
print(df_price_housing['furnishingstatus'].unique())

['yes' 'no']
['no' 'yes']
['no' 'yes']
['no' 'yes']
['yes' 'no']
[2 3 0 1]
['yes' 'no']
['furnished' 'semi-furnished' 'unfurnished']


In [4]:
#Clean and prepare data (transform yes to 1 and no to 0)
df_price_housing['mainroad'] = df_price_housing['mainroad'].replace({'yes':1,'no':0})
df_price_housing['guestroom'] = df_price_housing['guestroom'].replace({'yes':1,'no':0})
df_price_housing['basement'] = df_price_housing['basement'].replace({'yes':1,'no':0})
df_price_housing['hotwaterheating'] = df_price_housing['hotwaterheating'].replace({'yes':1,'no':0})
df_price_housing['airconditioning'] = df_price_housing['airconditioning'].replace({'yes':1,'no':0})
df_price_housing['prefarea'] = df_price_housing['prefarea'].replace({'yes':1,'no':0})
df_price_housing['furnishingstatus'] = df_price_housing['furnishingstatus'].replace({'furnished':2,'semi-furnished':1,'unfurnished':0})

df_price_housing

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,2
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,2
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,1
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,2
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,1,0,1,0,0,2,0,0
541,1767150,2400,3,1,1,0,0,0,0,0,0,0,1
542,1750000,3620,2,1,1,1,0,0,0,0,0,0,0
543,1750000,2910,3,1,1,0,0,0,0,0,0,0,2


In [5]:
# Set independent variables
X = df_price_housing[['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad',
       'guestroom', 'basement', 'hotwaterheating', 'airconditioning',
       'parking', 'prefarea', 'furnishingstatus']]

# Set dependent variable (price)
y = df_price_housing['price']

# Split data into training and testing sets (test size = 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Create a linear regression model
model = LinearRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

# Predicting on the test set
y_pred = model.predict(X_test)

In [6]:
# Evaluating the model
MSE = mean_squared_error(y_test, y_pred).round(4)
r2 = r2_score(y_test, y_pred).round(4)

# Model coefficients
coef = model.coef_.round(4)
intercept = model.intercept_.round(4)

# Linear equation for the model
equation = f"{intercept}"
for i, col in enumerate(X.columns):
  equation += f" + {coef[i]}*{col}"

print(f'1) MSE: {MSE}')
print(f'2) r2: {r2}')
print(f'3) Coefficients: { coef}')
print(f'4) Intercept: { intercept} \n')
print(f'ML regression: price ={equation} ')

1) MSE: 986041803890.027
2) r2: 0.6578
3) Coefficients: [2.53732200e+02 1.26430216e+05 1.00306572e+06 4.31159234e+05
 4.34983941e+05 3.95198712e+05 3.30288607e+05 1.20896581e+06
 8.59967238e+05 2.20086777e+05 6.93749764e+05 2.55656836e+05]
4) Intercept: -406907.4632 

ML regression: price =-406907.4632 + 253.7322*area + 126430.2161*bedrooms + 1003065.7209*bathrooms + 431159.2338*stories + 434983.9406*mainroad + 395198.7122*guestroom + 330288.6072*basement + 1208965.808*hotwaterheating + 859967.2382*airconditioning + 220086.7768*parking + 693749.7644*prefarea + 255656.8359*furnishingstatus 


In [7]:
# Predict the house price with random parameter
X_random = {
    'area':[6000] ,
    'bedrooms':[4] ,
    'bathrooms':[4] ,
    'stories':[2] ,
    'mainroad': [0],
    'guestroom': [1],
    'basement': [0],
    'hotwaterheating': [1],
    'airconditioning': [1],
    'parking': [2],
    'prefarea': [0],
    'furnishingstatus': [1]
}

X_random_data = pd.DataFrame(X_random)
predicted_price = model.predict(X_random_data).round(4)

print(f'Predicted price is: {predicted_price}')

Predicted price is: [9655750.3045]


In [8]:
# save model
pickle.dump(model, open('ml_6610422012.sav','wb') )

from google.colab import files
files.download('ml_6610422012.sav')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>